# Exercise 08

## Feature selection exercise with Titanic data




We'll be working with a dataset from Kaggle's Titanic competition: [data](https://github.com/justmarkham/DAT8/blob/master/data/titanic.csv), [data dictionary](https://www.kaggle.com/c/titanic/data)

**Goal**: Predict survival based on passenger characteristics

The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.


Read the data into Pandas

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/titanic.csv'
titanic = pd.read_csv(url, index_col='PassengerId')

titanic.Age.fillna(titanic.Age.median(), inplace=True)
titanic.loc[titanic.Embarked.isnull(), 'Embarked'] = titanic.Embarked.mode().values

titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)

titanic['Age2'] = titanic['Age'] ** 2
titanic['Age3'] = titanic['Age'] ** 3


features = ['Pclass', 'Age', 'Age2', 'Age3', 'Parch', 'SibSp', 'Sex_Female', 'Embarked_C', 'Embarked_Q', 'Embarked_S'] 
X = titanic[list(features)]
y = titanic['Survived']

## Exercise 8.1 (2 points)

Using the univariate selection method SelectKBest, which is the value of k that maximizes the accuracy of the model

In [16]:
results = pd.DataFrame(index=['k='+ str(i) for i in range(1, 11)], columns=['mean_accuracy'])

In [17]:
from sklearn.feature_selection import SelectKBest
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
t=[]



for i in range(1,11):
    sel = SelectKBest(k=i)
    sel.fit(X, y)
    sel.get_support()
    X_sel = sel.transform(X)
    results.loc['k='+str(i)] = pd.Series(cross_val_score(logreg, X_sel, y, cv=10, scoring='accuracy')).mean()

results

,mean_accuracy
k=1,0.786698
k=2,0.786698
k=3,0.772179
k=4,0.771055
k=5,0.776673
k=6,0.794627
k=7,0.798023
k=8,0.795813
k=9,0.71635
k=10,0.710732


# Exercise 8.2 (2 points)

Using the univariate selection method SelectPercentile, which is the value of percentile that maximizes the accuracy of the model

In [20]:
from sklearn.feature_selection import SelectPercentile, f_classif

results = pd.DataFrame(index=range(99), columns=['mean_accuracy'])

for i in range(1,100):
    sel = SelectPercentile(f_classif, percentile=i)
    sel.fit(X, y)
    sel.get_support()
    X_sel = sel.transform(X)
    results.iloc[i-1] = pd.Series(cross_val_score(logreg, X_sel, y, cv=10, scoring='accuracy')).mean()
    

In [21]:
results.idxmax()

mean_accuracy    66
dtype: int64

In [23]:
results.iloc[66]

mean_accuracy    0.798023
Name: 66, dtype: object

# Exercise 8.3 (3 points)

Using the recursive feature selection method RFE, which is the value of n_features_to_select that maximizes the accuracy of the model

In [26]:
from sklearn.feature_selection import RFE

results = pd.DataFrame(index=range(10), columns=['mean_accuracy'])

for i in range(1,11):
    sel = RFE(estimator=logreg, n_features_to_select=i)
    sel.fit(X, y)
    X_sel = sel.transform(X)
    results.iloc[i-1] = cross_val_score(logreg, X_sel, y, cv=10, scoring='accuracy').mean()

In [27]:
results

,mean_accuracy
0,0.786698
1,0.786698
2,0.772179
3,0.771055
4,0.784526
5,0.784526
6,0.783428
7,0.796912
8,0.801407
9,0.710732
